In [3]:
import requests
import json
import os
import pandas as pd
import credentials

In [4]:
baseurl = "https://www.ncdc.noaa.gov/cdo-web/api/v2/"
endpoint_data = "data"
custom_headers = {
    "token": credentials.token
}
datatypes = ["DLY-TAVG-NORMAL", "DLY-TMAX-NORMAL", "DLY-TMIN-NORMAL", "DLY-PRCP-50PCTL", "MTD-PRCP-NORMAL" ]
# extra_datatypes = ["EMNT", "EMXT", "EMSD", "EMXP"]  # trying out some extras -- look into them later
params = {
    "datasetid": "NORMAL_DLY",
    "datatypeid": datatypes,
    "startdate": "2010-01-01",
    "enddate": "2010-12-31",
    "limit": 1000,
    "units": "standard",
}

In [5]:
STATIONFN = "AK_NCDC_FirstOrderStations.json"
with open(STATIONFN, "r") as source:
    stationdata = json.load(source)

## Daily normals for stations

In [7]:
SAVEOUT = False
for station in stationdata:
    fullbase = requests.compat.urljoin(baseurl, endpoint_data)
    params["stationid"] = station["id"]
    params['datatypeid'] = datatypes
    print(station["shortname"])
    # we may have more results than our limit.
    results = []
    limit = 1000
    offset = 0
    while True:
        params['limit'] = limit
        params['offset'] = offset
        r = requests.get(
            fullbase, 
            headers=custom_headers,
            params=params,
        )
        response = json.loads(r.text)
        results.extend(response['results'])
        offset += limit
        if offset >= response['metadata']['resultset']['count']:
            break
    newdic = {}
    for result in results:
        try:
            newdic[result["date"][:10]][result["datatype"]] = result["value"]
        except KeyError:
            newdic[result["date"][:10]] = {}
            newdic[result["date"][:10]][result["datatype"]] = result["value"]
    dataforstation = pd.DataFrame.from_dict(newdic, orient="index")
    if SAVEOUT: 
        dataforstation.to_csv(
            "dailynormals_{}.dat".format(station["shortname"]),
            index_label="date", sep='\t'
    )

Fairbanks


In [8]:
newdic

{'2010-01-01': {'DLY-PRCP-50PCTL': 0.04,
  'DLY-TAVG-NORMAL': -6.8,
  'DLY-TMAX-NORMAL': 2.1,
  'DLY-TMIN-NORMAL': -15.7,
  'MTD-PRCP-NORMAL': 0.02},
 '2010-01-02': {'DLY-PRCP-50PCTL': 0.04,
  'DLY-TAVG-NORMAL': -7.0,
  'DLY-TMAX-NORMAL': 1.9,
  'DLY-TMIN-NORMAL': -15.9,
  'MTD-PRCP-NORMAL': 0.04},
 '2010-01-03': {'DLY-PRCP-50PCTL': 0.04,
  'DLY-TAVG-NORMAL': -7.2,
  'DLY-TMAX-NORMAL': 1.7,
  'DLY-TMIN-NORMAL': -16.1,
  'MTD-PRCP-NORMAL': 0.06},
 '2010-01-04': {'DLY-PRCP-50PCTL': 0.04,
  'DLY-TAVG-NORMAL': -7.4,
  'DLY-TMAX-NORMAL': 1.5,
  'DLY-TMIN-NORMAL': -16.3,
  'MTD-PRCP-NORMAL': 0.09},
 '2010-01-05': {'DLY-PRCP-50PCTL': 0.04,
  'DLY-TAVG-NORMAL': -7.5,
  'DLY-TMAX-NORMAL': 1.3,
  'DLY-TMIN-NORMAL': -16.4,
  'MTD-PRCP-NORMAL': 0.11},
 '2010-01-06': {'DLY-PRCP-50PCTL': 0.04,
  'DLY-TAVG-NORMAL': -7.7,
  'DLY-TMAX-NORMAL': 1.2,
  'DLY-TMIN-NORMAL': -16.6,
  'MTD-PRCP-NORMAL': 0.13},
 '2010-01-07': {'DLY-PRCP-50PCTL': 0.04,
  'DLY-TAVG-NORMAL': -7.9,
  'DLY-TMAX-NORMAL': 1.0,
  'DLY

In [29]:
params['datatypeid']

['DLY-TAVG-NORMAL',
 'DLY-TMAX-NORMAL',
 'DLY-TMIN-NORMAL',
 'DLY-PRCP-50PCTL',
 'MTD-PRCP-NORMAL']

Data examples below

In [30]:
results

[{'attributes': 'C',
  'datatype': 'DLY-PRCP-50PCTL',
  'date': '2010-01-01T00:00:00',
  'station': 'GHCND:USW00026411',
  'value': 0.04},
 {'attributes': 'C',
  'datatype': 'DLY-TAVG-NORMAL',
  'date': '2010-01-01T00:00:00',
  'station': 'GHCND:USW00026411',
  'value': -6.8},
 {'attributes': 'C',
  'datatype': 'DLY-TMAX-NORMAL',
  'date': '2010-01-01T00:00:00',
  'station': 'GHCND:USW00026411',
  'value': 2.1},
 {'attributes': 'C',
  'datatype': 'DLY-TMIN-NORMAL',
  'date': '2010-01-01T00:00:00',
  'station': 'GHCND:USW00026411',
  'value': -15.7},
 {'attributes': 'C',
  'datatype': 'MTD-PRCP-NORMAL',
  'date': '2010-01-01T00:00:00',
  'station': 'GHCND:USW00026411',
  'value': 0.02},
 {'attributes': 'C',
  'datatype': 'DLY-PRCP-50PCTL',
  'date': '2010-01-02T00:00:00',
  'station': 'GHCND:USW00026411',
  'value': 0.04},
 {'attributes': 'C',
  'datatype': 'DLY-TAVG-NORMAL',
  'date': '2010-01-02T00:00:00',
  'station': 'GHCND:USW00026411',
  'value': -7.0},
 {'attributes': 'C',
  'dat

In [31]:
dataforstation

,DLY-TAVG-NORMAL,DLY-TMIN-NORMAL,DLY-PRCP-50PCTL,DLY-TMAX-NORMAL,MTD-PRCP-NORMAL
2010-01-01,-6.8,-15.7,0.04,2.1,0.02
2010-01-02,-7.0,-15.9,0.04,1.9,0.04
2010-01-03,-7.2,-16.1,0.04,1.7,0.06
2010-01-04,-7.4,-16.3,0.04,1.5,0.09
2010-01-05,-7.5,-16.4,0.04,1.3,0.11
2010-01-06,-7.7,-16.6,0.04,1.2,0.13
2010-01-07,-7.9,-16.8,0.04,1.0,0.15
2010-01-08,-8.0,-16.9,0.04,0.8,0.17
2010-01-09,-8.2,-17.0,0.04,0.7,0.19
2010-01-10,-8.3,-17.2,0.04,0.6,0.21


In [56]:
results['results']

[{'attributes': 'C',
  'datatype': 'DLY-PRCP-50PCTL',
  'date': '2010-07-01T00:00:00',
  'station': 'GHCND:USW00026411',
  'value': 0.07},
 {'attributes': 'C',
  'datatype': 'DLY-TAVG-NORMAL',
  'date': '2010-07-01T00:00:00',
  'station': 'GHCND:USW00026411',
  'value': 63.6},
 {'attributes': 'C',
  'datatype': 'DLY-TMAX-NORMAL',
  'date': '2010-07-01T00:00:00',
  'station': 'GHCND:USW00026411',
  'value': 74.4},
 {'attributes': 'C',
  'datatype': 'DLY-TMIN-NORMAL',
  'date': '2010-07-01T00:00:00',
  'station': 'GHCND:USW00026411',
  'value': 52.8},
 {'attributes': 'C',
  'datatype': 'MTD-PRCP-NORMAL',
  'date': '2010-07-01T00:00:00',
  'station': 'GHCND:USW00026411',
  'value': 0.06},
 {'attributes': 'C',
  'datatype': 'DLY-PRCP-50PCTL',
  'date': '2010-07-02T00:00:00',
  'station': 'GHCND:USW00026411',
  'value': 0.07},
 {'attributes': 'C',
  'datatype': 'DLY-TAVG-NORMAL',
  'date': '2010-07-02T00:00:00',
  'station': 'GHCND:USW00026411',
  'value': 63.7},
 {'attributes': 'C',
  'dat

In [57]:
newdic = {}
for result in results:
    try:
        newdic[result["date"][:10]][result["datatype"]] = result["value"]
    except KeyError:
        newdic[result["date"][:10]] = {}
        newdic[result["date"][:10]][result["datatype"]] = result["value"]

In [58]:
newdic

{'2010-07-01': {'DLY-PRCP-50PCTL': 0.07,
  'DLY-TAVG-NORMAL': 63.6,
  'DLY-TMAX-NORMAL': 74.4,
  'DLY-TMIN-NORMAL': 52.8,
  'MTD-PRCP-NORMAL': 0.06},
 '2010-07-02': {'DLY-PRCP-50PCTL': 0.07,
  'DLY-TAVG-NORMAL': 63.7,
  'DLY-TMAX-NORMAL': 74.4,
  'DLY-TMIN-NORMAL': 52.9,
  'MTD-PRCP-NORMAL': 0.12},
 '2010-07-03': {'DLY-PRCP-50PCTL': 0.07,
  'DLY-TAVG-NORMAL': 63.7,
  'DLY-TMAX-NORMAL': 74.4,
  'DLY-TMIN-NORMAL': 53.0,
  'MTD-PRCP-NORMAL': 0.18},
 '2010-07-04': {'DLY-PRCP-50PCTL': 0.07,
  'DLY-TAVG-NORMAL': 63.7,
  'DLY-TMAX-NORMAL': 74.4,
  'DLY-TMIN-NORMAL': 53.0,
  'MTD-PRCP-NORMAL': 0.25},
 '2010-07-05': {'DLY-PRCP-50PCTL': 0.07,
  'DLY-TAVG-NORMAL': 63.7,
  'DLY-TMAX-NORMAL': 74.4,
  'DLY-TMIN-NORMAL': 53.0,
  'MTD-PRCP-NORMAL': 0.31},
 '2010-07-06': {'DLY-PRCP-50PCTL': 0.07,
  'DLY-TAVG-NORMAL': 63.7,
  'DLY-TMAX-NORMAL': 74.3,
  'DLY-TMIN-NORMAL': 53.1,
  'MTD-PRCP-NORMAL': 0.37},
 '2010-07-07': {'DLY-PRCP-50PCTL': 0.07,
  'DLY-TAVG-NORMAL': 63.6,
  'DLY-TMAX-NORMAL': 74.2,
  'DL

In [59]:
pd.DataFrame.from_dict(newdic, orient="index")

,DLY-TMIN-NORMAL,DLY-PRCP-50PCTL,DLY-TMAX-NORMAL,DLY-TAVG-NORMAL,MTD-PRCP-NORMAL
2010-07-01,52.8,0.07,74.4,63.6,0.06
2010-07-02,52.9,0.07,74.4,63.7,0.12
2010-07-03,53.0,0.07,74.4,63.7,0.18
2010-07-04,53.0,0.07,74.4,63.7,0.25
2010-07-05,53.0,0.07,74.4,63.7,0.31
2010-07-06,53.1,0.07,74.3,63.7,0.37
2010-07-07,53.1,0.07,74.2,63.6,0.43
2010-07-08,53.0,0.08,74.2,63.6,0.50
2010-07-09,53.0,0.07,74.1,63.5,0.56
2010-07-10,53.0,0.07,74.0,63.5,0.63


## Monthly normals for stations

In [77]:
SAVEOUT = True
for station in stationdata:
    fullbase = requests.compat.urljoin(baseurl, endpoint_data)
    print(station['name'])
    params["stationid"] = station["id"]
    params["datasetid"] = "NORMAL_MLY"
    params["datatypeid"] = [
        'MLY-PRCP-NORMAL', 
        'MLY-SNOW-NORMAL', 
        'MLY-TMIN-NORMAL', 
        'MLY-TAVG-NORMAL', 
        'MLY-TMAX-NORMAL']
    r = requests.get(
        fullbase, 
        headers=custom_headers,
        params=params,
    )
    results = json.loads(r.text)
    newdic = {}
    for result in results['results']:
        try:
            newdic[result["date"][:10]][result["datatype"]] = result["value"]
        except KeyError:
            newdic[result["date"][:10]] = {}
            newdic[result["date"][:10]][result["datatype"]] = result["value"]
    dataforstation = pd.DataFrame.from_dict(newdic, orient="index")
    if SAVEOUT: 
        dataforstation.to_csv(
            "monthlynormals_{}.dat".format(station["shortname"]),
            index_label="date", sep='\t'
    )

FAIRBANKS INTERNATIONAL AIRPORT, AK US
ANNETTE WEATHER SERVICE OFFICE AIRPORT, AK US
JUNEAU AIRPORT, AK US
YAKUTAT AIRPORT, AK US
KODIAK AIRPORT, AK US
KING SALMON AIRPORT, AK US
HOMER AIRPORT, AK US
COLD BAY AIRPORT, AK US
ANCHORAGE TED STEVENS INTERNATIONAL AIRPORT, AK US
ST PAUL ISLAND AIRPORT, AK US
BIG DELTA AIRPORT, AK US
GULKANA AIRPORT, AK US
VALDEZ WEATHER SERVICE OFFICE, AK US
MCGRATH AIRPORT, AK US
TALKEETNA AIRPORT, AK US
BETTLES AIRPORT, AK US
BETHEL AIRPORT, AK US
KOTZEBUE RALPH WEIN MEMORIAL AIRPORT, AK 
NOME MUNICIPAL AIRPORT, AK US
BARROW W POST W ROGERS AIRPORT, AK US


In [75]:
dataforstation

,MLY-TAVG-NORMAL,MLY-TMAX-NORMAL,MLY-TMIN-NORMAL,MLY-PRCP-NORMAL,MLY-SNOW-NORMAL
2010-01-01,-7.9,1.1,-16.9,0.58,10.3
2010-02-01,-1.3,10.0,-12.7,0.42,8.1
2010-03-01,11.4,25.4,-2.5,0.25,4.9
2010-04-01,32.5,44.5,20.6,0.31,2.9
2010-05-01,49.4,61.0,37.8,0.60,0.9
2010-06-01,60.4,71.6,49.3,1.37,0.0
2010-07-01,62.5,72.7,52.3,2.16,0.0
2010-08-01,56.1,65.9,46.4,1.88,-777.7
2010-09-01,44.9,54.6,35.1,1.10,1.8
2010-10-01,24.2,31.9,16.5,0.83,10.8
